In [4]:
import pandas as pd
import os

# Path to the raw data folder
RAW_DATA_PATH = os.path.join('data', 'raw')
PROCESSED_DATA_PATH = os.path.join('data', 'processed')


In [8]:
import pandas as pd
import os

# Path to the raw data folder
RAW_DATA_PATH = os.path.join('data', 'raw')
PROCESSED_DATA_PATH = os.path.join('data', 'processed')

def load_raw_data(file_name):
    file_path = os.path.join(RAW_DATA_PATH, file_name)
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_name} not found in {RAW_DATA_PATH}")
    return pd.read_csv(file_path)

def remove_columns(data):
    columns_to_drop = ['Iso2', 'Iso3', 'Indicator', 'Unit', 'Source', 
                       'Cts_Code', 'Cts_Name', 'Cts_Full_Descriptor']
    data = data.drop(columns=columns_to_drop, errors='ignore')
    return data

def filter_years(data):
    # Extract columns for years from F1961 to F2022
    year_columns = [f'F{i}' for i in range(1961, 2023)]
    data = data[['Country'] + year_columns]  # Include 'Country' and year columns
    return data

def remove_duplicates(data):
    before = data.shape[0]
    data = data.drop_duplicates()
    after = data.shape[0]
    print(f"Removed {before - after} duplicate rows.")
    return data

def handle_missing_values(data):
    # Display columns with missing data
    missing = data.isnull().sum()
    print(f"Missing values:\n{missing[missing > 0]}")

    # Example: Fill missing values in a specific column with the mean
    if 'Indicator Value' in data.columns:
        data['Indicator Value'] = data['Indicator Value'].fillna(data['Indicator Value'].mean())

    # Drop rows with missing essential columns
    data = data.dropna(subset=['Country'])
    print(f"Remaining missing values:\n{data.isnull().sum().sum()}")
    return data

def save_cleaned_data(data, file_name):
    os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)
    file_path = os.path.join(PROCESSED_DATA_PATH, file_name)
    data.to_csv(file_path, index=False)
    print(f"Cleaned data saved to {file_path}")

if __name__ == "__main__":
    # Load raw data
    raw_data = load_raw_data('climate_change_indicators.csv')
    
    # Clean up column names (strip spaces and standardize case)
    raw_data.columns = raw_data.columns.str.strip().str.title()
    
    # Print the columns to check if 'Country' exists
    print("Columns in dataset:", raw_data.columns)

    # Filter for the required columns
    filtered_data = filter_years(raw_data)
    
    # Remove unwanted columns
    filtered_data = remove_columns(filtered_data)
    
    # Apply transformations
    filtered_data = remove_duplicates(filtered_data)
    filtered_data = handle_missing_values(filtered_data)
    
    # Save the cleaned data
    save_cleaned_data(filtered_data, 'climate_change_indicators_cleaned.csv')


Columns in dataset: Index(['Objectid', 'Country', 'Iso2', 'Iso3', 'Indicator', 'Unit', 'Source',
       'Cts_Code', 'Cts_Name', 'Cts_Full_Descriptor', 'F1961', 'F1962',
       'F1963', 'F1964', 'F1965', 'F1966', 'F1967', 'F1968', 'F1969', 'F1970',
       'F1971', 'F1972', 'F1973', 'F1974', 'F1975', 'F1976', 'F1977', 'F1978',
       'F1979', 'F1980', 'F1981', 'F1982', 'F1983', 'F1984', 'F1985', 'F1986',
       'F1987', 'F1988', 'F1989', 'F1990', 'F1991', 'F1992', 'F1993', 'F1994',
       'F1995', 'F1996', 'F1997', 'F1998', 'F1999', 'F2000', 'F2001', 'F2002',
       'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008', 'F2009', 'F2010',
       'F2011', 'F2012', 'F2013', 'F2014', 'F2015', 'F2016', 'F2017', 'F2018',
       'F2019', 'F2020', 'F2021', 'F2022'],
      dtype='object')
Removed 0 duplicate rows.
Missing values:
F1961    37
F1962    36
F1963    37
F1964    37
F1965    37
         ..
F2018    12
F2019    12
F2020    13
F2021    12
F2022    12
Length: 62, dtype: int64
Remaining mis